# ETF TIGER 미국나스닥 100 일별시세 데이터 수집하기
(네이버 금융 웹페이지)

## 1. 웹스크랩핑

In [3]:
import pandas as pd 
import requests 
from bs4 import BeautifulSoup as bs 
import tqdm

In [33]:
url = 'https://finance.naver.com/item/sise_day.nhn?code=133690&page=1'
headers = {'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36'}
response = requests.get(url, headers = headers )
soup = bs(response.text, 'lxml')
temp= soup.select('table')
table = pd.read_html(str(temp))
day_temp =table[0].dropna()
day_temp

,날짜,종가,전일비,시가,고가,저가,거래량
1,2021.07.01,73980.0,330.0,73845.0,74095.0,73790.0,61573.0
2,2021.06.30,73650.0,295.0,73750.0,74050.0,73605.0,99353.0
3,2021.06.29,73355.0,555.0,73385.0,73495.0,73330.0,65044.0
4,2021.06.28,72800.0,150.0,72650.0,72815.0,72570.0,52572.0
5,2021.06.25,72650.0,190.0,72975.0,73010.0,72565.0,96230.0
9,2021.06.24,72840.0,75.0,72735.0,72975.0,72705.0,91304.0
10,2021.06.23,72915.0,1110.0,72445.0,72915.0,72445.0,89395.0
11,2021.06.22,71805.0,250.0,71800.0,71940.0,71730.0,64545.0
12,2021.06.21,71555.0,460.0,71500.0,71555.0,71245.0,84238.0
13,2021.06.18,72015.0,1495.0,71010.0,72135.0,71005.0,189338.0


## 2. page별 수집 함수 만들기

In [44]:
def day_temp(item_code, page_no):
    url = f'https://finance.naver.com/item/sise_day.nhn?code={item_code}&page={page_no}'
    headers = {'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36'}
    response = requests.get(url, headers = headers )
    soup = bs(response.text, 'lxml')
    temp= soup.select('table')
    table = pd.read_html(str(temp))
    day_temp =table[0].dropna()
    return day_temp

In [45]:
item_code = 133690 
page_no = 5
day_temp(item_code, page_no)

# 정상 동작함

,날짜,종가,전일비,시가,고가,저가,거래량
1,2021.05.04,69280.0,550.0,69830.0,69830.0,69085.0,77163.0
2,2021.05.03,69830.0,450.0,69420.0,69830.0,69420.0,61512.0
3,2021.04.30,69380.0,260.0,69195.0,69380.0,69020.0,60086.0
4,2021.04.29,69640.0,10.0,69570.0,69720.0,69415.0,64746.0
5,2021.04.28,69720.0,180.0,69785.0,69845.0,69630.0,58954.0
9,2021.04.27,69900.0,355.0,69865.0,70000.0,69765.0,79738.0
10,2021.04.26,69545.0,575.0,69625.0,69725.0,69455.0,61963.0
11,2021.04.23,68970.0,910.0,69130.0,69260.0,68955.0,106331.0
12,2021.04.22,69880.0,825.0,69510.0,69880.0,69510.0,51768.0
13,2021.04.21,69055.0,445.0,69000.0,69055.0,68705.0,82281.0


## 3. for문으로 원하는 페이지까지 수집

In [50]:
# 서버에 부담을 주지 않기위해 time.sleep() 적용

import time 

start_no =1
end_no = int(input('수집하고자 하는 페이지.no 입력:'))
day_temp_list=[]

for page_no in range(start_no,end_no +1 ):
    a = day_temp(item_code, page_no)
    day_temp_list.append(a)
    time.sleep

수집하고자 하는 페이지.no 입력:12


In [51]:
len(day_temp_list)

12

## 4. concat 으로 테이블 하나로 합치기

In [58]:
df = pd.concat(day_temp_list)
df.head()

,날짜,종가,전일비,시가,고가,저가,거래량
1,2021.07.01,73980.0,330.0,73845.0,74095.0,73790.0,61573.0
2,2021.06.30,73650.0,295.0,73750.0,74050.0,73605.0,99353.0
3,2021.06.29,73355.0,555.0,73385.0,73495.0,73330.0,65044.0
4,2021.06.28,72800.0,150.0,72650.0,72815.0,72570.0,52572.0
5,2021.06.25,72650.0,190.0,72975.0,73010.0,72565.0,96230.0


## 5. 코드로 종목명 가져오기

In [115]:
df_krx = pd.read_csv('krx.csv')
df_krx.head(2)

,Symbol,Market,Name,Sector,Industry,ListingDate,SettleMonth,Representative,HomePage,Region
0,060310,KOSDAQ,3S,특수 목적용 기계 제조업,반도체 웨이퍼 캐리어,2002-04-23,03월,"박종익, 김세완 (각자 대표이사)",http://www.3sref.com,서울특별시
1,095570,KOSPI,AJ네트웍스,산업용 기계 및 장비 임대업,"렌탈(파렛트, OA장비, 건설장비)",2015-08-21,12월,박대현,http://www.ajnet.co.kr,서울특별시


In [143]:
item_code = '133690'
name = df_krx[df_krx['Symbol']== item_code]['Name']
name

2071    TIGER 미국나스닥100
Name: Name, dtype: object

## 6. 종목명, 종목코드 컬럼 추가하기

In [144]:
item_code = 133690
item_name = 'TIGER 미국나스닥100'
df['종목코드'] = item_code
df['종목명']= 'TIGER 미국나스닥100'
df=df.drop('전일비', axis = 1)
df

KeyError: "['전일비'] not found in axis"

In [120]:
cols = ['종목명', '종목코드', '날짜', '종가', '시가', '고가', '저가', '거래량']
cols

['종목명', '종목코드', '날짜', '종가', '시가', '고가', '저가', '거래량']

In [122]:
df = df[cols]
df

,종목명,종목코드,날짜,종가,시가,고가,저가,거래량
1,TIGER 미국나스닥100,133690,2021.07.01,73980.0,73845.0,74095.0,73790.0,61573.0
2,TIGER 미국나스닥100,133690,2021.06.30,73650.0,73750.0,74050.0,73605.0,99353.0
3,TIGER 미국나스닥100,133690,2021.06.29,73355.0,73385.0,73495.0,73330.0,65044.0
4,TIGER 미국나스닥100,133690,2021.06.28,72800.0,72650.0,72815.0,72570.0,52572.0
5,TIGER 미국나스닥100,133690,2021.06.25,72650.0,72975.0,73010.0,72565.0,96230.0
...,...,...,...,...,...,...,...,...
9,TIGER 미국나스닥100,133690,2021.01.14,63795.0,63510.0,64170.0,63510.0,109451.0
10,TIGER 미국나스닥100,133690,2021.01.13,63480.0,63395.0,63530.0,63200.0,98183.0
11,TIGER 미국나스닥100,133690,2021.01.12,63705.0,63685.0,63780.0,63540.0,133121.0
12,TIGER 미국나스닥100,133690,2021.01.11,64110.0,64100.0,64320.0,64085.0,246669.0


## 파일 저장하고 불러오기

In [151]:
date = df.iloc[0,2].replace('.','-')
date

'2021-07-01'

In [152]:
file_name = f'{date}-{item_name}.csv'
file_name

'2021-07-01-TIGER 미국나스닥100.csv'

In [153]:
df.to_csv(file_name, index=False)

In [154]:
pd.read_csv(file_name)

,종목명,종목코드,날짜,종가,시가,고가,저가,거래량
0,TIGER 미국나스닥100,133690,2021.07.01,73980.0,73845.0,74095.0,73790.0,61573.0
1,TIGER 미국나스닥100,133690,2021.06.30,73650.0,73750.0,74050.0,73605.0,99353.0
2,TIGER 미국나스닥100,133690,2021.06.29,73355.0,73385.0,73495.0,73330.0,65044.0
3,TIGER 미국나스닥100,133690,2021.06.28,72800.0,72650.0,72815.0,72570.0,52572.0
4,TIGER 미국나스닥100,133690,2021.06.25,72650.0,72975.0,73010.0,72565.0,96230.0
...,...,...,...,...,...,...,...,...
115,TIGER 미국나스닥100,133690,2021.01.14,63795.0,63510.0,64170.0,63510.0,109451.0
116,TIGER 미국나스닥100,133690,2021.01.13,63480.0,63395.0,63530.0,63200.0,98183.0
117,TIGER 미국나스닥100,133690,2021.01.12,63705.0,63685.0,63780.0,63540.0,133121.0
118,TIGER 미국나스닥100,133690,2021.01.11,64110.0,64100.0,64320.0,64085.0,246669.0
